In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import os
from urllib2 import urlopen
import urllib
import requests
from pymongo import MongoClient
import json#for storing beautiful soup to mongo

In [2]:
client = MongoClient()

In [3]:
img_db = client['images']
image_data = img_db['image_data']

In [4]:
#To get a beautiful soup object
def make_soup(url):
    
    #html = urlopen(url).read()
    page = requests.get(url)
    html = page.text
    return BeautifulSoup(html, 'html.parser')


In [5]:
#Returns the product specification table FROM amazon product page
def get_prod_table(soup):
    tab = soup.select('table.a-keyvalue.a-spacing-mini')[0] # Because ut returns 1 table in a list
    #print tab
    table = [] 
    for tr in tab.findAll('tr'):
        try:
            table.append(str((tr.th.get_text().strip(),tr.td.get_text().strip())))
        except:
            pass
    return table        

In [6]:
#Returns the product description FROM amazon product page
def get_prod_desc(soup):
    desc = soup.findAll('div',attrs={'id' : 'productDescription'})[0]#all product description are kept like this
    return desc.get_text().strip()

In [7]:
#Returns the title of the amazon product page
def get_prod_title(soup):
    title = soup.findAll('span',attrs={'id' : 'productTitle'})[0]
    return str(title.get_text().strip())  

In [8]:
#INsert data into mongodb

def insert_to_mongo(soup,prod_url,img_url,link_counter):
    
    #Get decription table and product description
    table = get_prod_table(soup)
    
    description = get_prod_desc(soup)
    title = get_prod_title(soup)
    
    image_data.insert_one({ 'img_no' : link_counter,'prod_url':prod_url,'img_url': img_url,"prod_info_table":table,"prod_desc": description, "title" :title  })


In [9]:
# Given a link and the number of link creates a folder using number and stores images in it   
#Link counter counts each page images are scraped from used to save the images
def get_images(prod_url, link_counter):  
    
    #Make soup with product url
    soup = make_soup(prod_url)
    
    #this makes a list of bs4 element image tags
    images = [img for img in soup.select('img.a-dynamic-image.a-stretch-vertical')]
    
    #print (str(len(images)) + "images found.")
    #print 'Downloading images to current working directory.'
    #compile our unicode list of image links
    
    image_links = [each['data-old-hires'] for each in images]
    
    #print image_links
    #To standardize the name of each image on prod_url we use file_name as a counter
    file_name = 0
    
    #Create a sub directory for each link and go to that directory
    os.mkdir('{0}'.format(link_counter))
    os.chdir('{0}'.format(link_counter))
    
    
    
    #Get images and store them 
    for each in image_links:
        try:
            filename='{0}.jpg'.format(link_counter)
            #print "???????????????", filename
            #print each
            
            urllib.urlretrieve(each, filename)
            #Alternative to urlretrieve
            #r = requests.get(each)
            #with open(filename,'wb') as f:
            #f.write(r.content)
            file_name  += 1
            #insert text to mongodb
            insert_to_mongo(soup,prod_url,each,link_counter)
            
        except:
            pass
        
    # return to original directory
    os.chdir('..')
    return image_links

In [10]:

def lets_scrape_amazon(init_url,no_of_pages, prog_track_interval = 10):

    link_counter  = 0

    for i in range(1,(no_of_pages+1)):
        # Link for wristwatches with rating > 4 stars
        url =init_url.format(i,i)
        
        #Make soup with the search page
        soup = make_soup(url)
        
        #Get the individual links to the product for each search page
        links = [link.get('href') for link in soup.select("a.a-link-normal.s-access-detail-page")]
        
        for link in links:
            get_images(link,link_counter)
            #Printing to track progress
            if (link_counter% prog_track_interval) == 0 :
                print "%d images downloaded"%(link_counter)
            link_counter += 1 
        

In [11]:
x = requests.get('http://www.amazon.com/Casio-MQ24-1E-Black-Resin-Watch/dp/B000GAWSHM/ref=pd_sim_241_2?ie=UTF8&dpID=41ISNYs3UcL&dpSrc=sims&preST=_AC_UL160_SR86%2C160_&refRID=1AG5VNPHYQX7XQ4KXRPX')

In [12]:
soup =  BeautifulSoup(x.text,'html.parser')

#Code Test

link_counter  = 0

for i in range(1,100):
    # Link for wristwatches with rating > 4 stars
    url ="http://www.amazon.com/s/ref=sr_pg_{0}?fst=as%3Aoff&rh=n%3A7141123011%2Cn%3A10445813011%2Cn%3A7147441011%2Cn%3A6358539011%2Cn%3A6358540011%2Ck%3Awrist+watches%2Cp_72%3A2661618011&page={1}&bbn=10445813011&keywords=wrist+watches&ie=UTF8&qid=1451726453".format(i,i)
    soup = make_soup(url)
    links = [link.get('href') for link in soup.select("a.a-link-normal.s-access-detail-page")]
    link_counter +=1
    for link in links:
        print ">>>>>>%s>>>>>"%link_counter,link
        
    #img_urls = [] Can be used to save urls. We can also get titles fr each of the images describing what they are
    

In [13]:
watch_search_url = "http://www.amazon.com/s/ref=sr_pg_{0}?fst=as%3Aoff&rh=n%3A7141123011%2Cn%3A10445813011%2Cn%3A7147441011%2Cn%3A6358539011%2Cn%3A6358540011%2Ck%3Awrist+watches%2Cp_72%3A2661618011&page={1}&bbn=10445813011&keywords=wrist+watches&ie=UTF8&qid=1451726453"

In [14]:
lets_scrape_amazon(watch_search_url,100,20)

0 images downloaded
20 images downloaded
40 images downloaded
60 images downloaded
80 images downloaded
100 images downloaded
120 images downloaded
140 images downloaded
160 images downloaded
180 images downloaded
200 images downloaded
220 images downloaded
240 images downloaded
260 images downloaded
280 images downloaded
300 images downloaded
320 images downloaded
340 images downloaded
360 images downloaded
380 images downloaded
400 images downloaded
420 images downloaded
440 images downloaded
460 images downloaded
480 images downloaded
500 images downloaded
520 images downloaded
540 images downloaded
560 images downloaded
580 images downloaded
600 images downloaded
620 images downloaded
640 images downloaded
660 images downloaded
680 images downloaded
700 images downloaded
720 images downloaded
740 images downloaded
760 images downloaded
780 images downloaded
800 images downloaded
820 images downloaded
840 images downloaded
860 images downloaded
880 images downloaded
900 images downl

In [103]:
!pwd

/Users/Iskandar/.Trash/5 1.47.11 AM/1
